In [1]:
#Installing Libraries

!pip install yfinance
!pip install matplotlib
!pip install pandas
!pip install finnhub-python

In [2]:
#Importing Libraries

#Website 1: https://finnhub.io/docs/api
#Website 2: https://finance.yahoo.com/quote/%5EIPO/history/

import numpy as np
import pandas as pd
import requests
import yfinance as yf
import pandas_datareader as pdr
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

import matplotlib.pyplot as plt
import datetime

In [3]:
#Key to use Finnhub Data

Key = 'ctavj11r01qgsps8306gctavj11r01qgsps83070'

In [4]:
#Importing Finnhub Data

import finnhub
finnhub_client = finnhub.Client(api_key="ctavj11r01qgsps8306gctavj11r01qgsps83070")

res_ipo = (finnhub_client.ipo_calendar(_from="2022-01-01", to="2023-12-31"))

print(res_ipo)

{'ipoCalendar': [{'date': '2023-12-29', 'exchange': None, 'name': 'Epsium Enterprise Ltd', 'numberOfShares': None, 'price': None, 'status': 'filed', 'symbol': 'EPSM', 'totalSharesValue': 10000000}, {'date': '2023-12-29', 'exchange': None, 'name': 'LAFAYETTE ENERGY CORP.', 'numberOfShares': None, 'price': None, 'status': 'filed', 'symbol': 'LEC', 'totalSharesValue': 6900000}, {'date': '2023-12-28', 'exchange': 'NASDAQ Capital', 'name': 'NB Bancorp, Inc.', 'numberOfShares': 40997500, 'price': '10.00', 'status': 'priced', 'symbol': 'NBBK', 'totalSharesValue': 409975000}, {'date': '2023-12-27', 'exchange': 'NASDAQ Global', 'name': 'Iron Horse Acquisitions Corp.', 'numberOfShares': 6100000, 'price': '10.00', 'status': 'priced', 'symbol': 'IROHU', 'totalSharesValue': 61000000}, {'date': '2023-12-27', 'exchange': 'NASDAQ Capital', 'name': 'Docola, Inc.', 'numberOfShares': 1060870, 'price': '5.75-6.75', 'status': 'expected', 'symbol': 'DOCO', 'totalSharesValue': 8235000}, {'date': '2023-12-27'

In [5]:
#First View of Data

df = pd.DataFrame(res_ipo['ipoCalendar'])
df.head(10)

,date,exchange,name,numberOfShares,price,status,symbol,totalSharesValue
0,2023-12-29,None,Epsium Enterprise Ltd,NaN,None,filed,EPSM,10000000.0
1,2023-12-29,None,LAFAYETTE ENERGY CORP.,NaN,None,filed,LEC,6900000.0
2,2023-12-28,NASDAQ Capital,"NB Bancorp, Inc.",40997500.0,10.00,priced,NBBK,409975000.0
3,2023-12-27,NASDAQ Global,Iron Horse Acquisitions Corp.,6100000.0,10.00,priced,IROHU,61000000.0
4,2023-12-27,NASDAQ Capital,"Docola, Inc.",1060870.0,5.75-6.75,expected,DOCO,8235000.0
5,2023-12-27,NASDAQ Capital,ZEROSPO,1500000.0,4.00-5.00,expected,ZRSP,8625000.0
6,2023-12-22,None,Micropolis Holding Co,NaN,None,filed,,47150000.0
7,2023-12-22,None,LZ Technology Holdings Ltd,NaN,None,filed,LZMH,60000000.0
8,2023-12-22,None,Chun Hui Le Wan International Holding Group Ltd,NaN,None,filed,CHLW,1000000.0
9,2023-12-21,None,3 E Network Technology Group Ltd,NaN,None,filed,EEET,20700000.0


In [6]:
# The highest price within a day per Yahoo Finance

import yfinance as yf

def get_latest_high(stock_symbol):
  try:
    ticker = yf.Ticker(stock_symbol)
    today_data = ticker.history(period='1d')
    return today_data['High'].iloc[0]
  except:
    return np.nan

In [7]:
# The lastest total revenue per Yahoo Finance

import yfinance as yf
import numpy as np

def get_latest_revenue(stock_symbol):

    try:
        ticker = yf.Ticker(stock_symbol)
        financials = ticker.financials
        if 'Total Revenue' in financials.index:
            latest_revenue = financials.loc['Total Revenue'].dropna().iloc[0]
            return latest_revenue
        else:
            return np.nan

    except Exception as e:
        return np.nan

In [8]:
# The lastest operating expense per Yahoo Finance

import yfinance as yf
import numpy as np

def get_latest_operating_expense(stock_symbol):
    try:
        ticker = yf.Ticker(stock_symbol)

        financials = ticker.financials

        if 'Operating Expense' in financials.index:
            latest_operating_expense = financials.loc['Operating Expense'].dropna().iloc[0]
            return latest_operating_expense
        else:
            return np.nan

    except Exception as e:
        return np.nan

In [9]:
#Industry Classification per Yahoo Finance

import yfinance as yf
import numpy as np

def get_industry(stock_symbol):
    try:
        ticker = yf.Ticker(stock_symbol)

        stock_info = ticker.info

        industry = stock_info.get('industry', np.nan)

        return industry

    except Exception as e:
        return np.nan

In [ ]:
#Creating new column and obtaining the relevant data by stock symbol

df['Latest Price'] = df['symbol'].apply(get_latest_high)
df['Revenue'] = df['symbol'].apply(get_latest_revenue)
df['Op Expense'] = df['symbol'].apply(get_latest_operating_expense)
df['Industry'] = df['symbol'].apply(get_industry)


ERROR:yfinance:$EPSM: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


In [ ]:
df

In [ ]:
# Data Cleaning

# Removing rows where price is a range
df['price'] = pd.to_numeric(df['price'], errors = 'coerce')

# Changing the remaining values to float
df [['price', 'Latest Price']] = df  [['price', 'Latest Price']].astype(float)

#Renaming Columns
df ['Percent Increase'] = ((df['Latest Price'] - df['price']) / df['price']) * 100
df ['IPO Date'] = df ['date']
df ['Stock Exchange'] = df ['exchange']
df ['Company Name'] = df ['name']
df ['Total Shares Issued'] = df ['numberOfShares']
df ['IPO Price'] = df ['price']
df ['Ticker'] = df ['symbol']
df ['Latest Revenue'] = df ['Revenue'].apply(lambda x: f"{x:.2f}")
df ['Latest Op Expense'] = df ['Op Expense'].apply(lambda x: f"{x:.2f}")

#Rearranging columnns (and also filtering out ones that are needed)
new_order = ['IPO Date', 'Ticker', 'Company Name', 'Industry','IPO Price', 'Total Shares Issued', 'Latest Price', 'Percent Increase', 'Latest Revenue', 'Latest Op Expense']
df = df.reindex(columns=new_order)

#Dropping NaN from the Percent Increase Column
df.dropna(subset=['Percent Increase'], inplace=True)

#Sorting by descending value based on Percent Increase
df.sort_values('Percent Increase', ascending = False)

df

In [ ]:
# Statistical Data

avg_p_inc_by_industry = df.groupby('Industry')['Percent Increase'].agg(
    Count = 'count',
    Min_Increase ='min',
    Max_Increase ='max',
    Range = lambda x: x.max() - x.min(),
    Standard_Deviation ='std',
    Average ='mean'
).reset_index()

avg_p_inc_by_industry

# Biotech & Shell Companies being the most common industry for IPOs
# Regional banks being the best investments for this year with all tickers experiencing growth

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a boxplot
plt.figure(figsize=(75, 30))
sns.boxplot(x='Industry', y='Percent Increase', data=df)

# Customize the boxplot
plt.title('Box Plot of Percent Increase by Industry', fontsize=100)
plt.xlabel('Industry', fontsize=100)
plt.ylabel('Percent Increase', fontsize=100)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()





In [ ]:
# Correlation Analysis
clean_corr = df.dropna(subset=['Percent Increase', 'Latest Op Expense','Latest Revenue'])

corr1 = clean_corr['Latest Op Expense'].corr(df['Percent Increase'])
corr2 = clean_corr['Latest Op Expense'].corr(df['Latest Price'])
corr3 = clean_corr['Latest Revenue'].corr(df['Percent Increase'])

print(f"Correlation between Expenses and Percent Increase: {corr1.round(4)}")
print(f"Correlation between Expenses and Latest Price: {corr2.round(4)}")
print(f"Correlation between Latest Revenue and Percent Increase: {corr3.round(4)}")

In [ ]:
# Most Successful Month for IPOs

df['IPO Date'] = pd.to_datetime(df['IPO Date'])
df['Month'] = df['IPO Date'].dt.month

cleaned_month = df.dropna(subset=['Month', 'Percent Increase'])

Avg_by_Month = cleaned_month.groupby('Month')['Percent Increase'].mean()

Avg_by_Month

# Best month for IPO is June

In [ ]:
import matplotlib.pyplot as plt

# make all months present
all_months = range(1, 13)
Avg_by_Month = Avg_by_Month.reindex(all_months, fill_value=0)

# make a bar chart
plt.figure(figsize=(10, 5))
Avg_by_Month.plot(kind='bar')

# Customize the bar chart
plt.title('Average Percent Increase by IPO Month', fontsize=18)
plt.xlabel('Months', fontsize=11)
plt.ylabel('Average Percent Increase', fontsize=11)
plt.xticks(ticks=range(12), labels=[
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)
plt.grid(axis='y', linestyle='-')
plt.tight_layout()

plt.show()

In [ ]:
# Top 20% of companies based on IPO performance and total % increase

percentile_80 = df['Percent Increase'].quantile(0.8) # identifies the % increase number for the 80th percentile

top_20p = df[df['Percent Increase'] > percentile_80] # companies with greater than the 80th percentile / focus on the upper tail

top_20p_industry = top_20p.sort_values(by=['Industry', 'Percent Increase'], ascending=[True, False]) # sort by industry & percent increase

top_20p_industry

# Top industries seem to be regional banks and software.

In [ ]:

# Count frequency of each industry in the filtered and sorted DataFrame
industry_counts = top_20p_industry['Industry'].value_counts()

# Plotting the industry frequency as a bar chart
plt.figure(figsize=(15,7))
industry_counts.plot(kind='bar', edgecolor='black')
plt.title('Frequency of Top Industries (Top 20% IPO Performance)')
plt.xlabel('Industry')
plt.ylabel('Frequency')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
top_20p_industry_cleaned = top_20p_industry.dropna(subset=['Latest Revenue', 'Latest Op Expense'])

plt.scatter(
    top_20p_industry_cleaned['Latest Op Expense'] / 1e6,
    top_20p_industry_cleaned['Latest Revenue'] / 1e6,
    alpha=0.7,
    edgecolor='black'
)

plt.title('Scatter Plot of Revenue vs Expense', fontsize=14)
plt.xlabel('Latest Operating Expenses', fontsize=12)
plt.ylabel('Latest Revenue', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='Industry', y='Percent Increase', data=top_20p_industry)

# Customize the boxplot
plt.title('Box Plot of Percent Increase by Industry 20 Percentile')
plt.xlabel('Industry')
plt.ylabel('Percent Increase')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


In [ ]:
print(df.columns)

In [ ]:
# Database Insert
import sqlite3
from google.colab import files

# Connection to IPO_Database database
conn = sqlite3.connect('IPO_Database.db')

# Allows us to manipulate the database with sql commands
cursor = conn.cursor()

# Done to standardize the column names for SQL inserts
df.columns = [col.replace(' ', '_') for col in df.columns]

# SQL Query to create the table for our data.
query = ("""
CREATE TABLE IF NOT EXISTS IPO_Database (
    IPO_Date TEXT,
    Ticker TEXT,
    Company_Name TEXT,
    Industry TEXT,
    IPO_Price REAL,
    Total_Shares_Issued REAL,
    Latest_Price REAL,
    Percent_Increase REAL
    Latest_Revenue REAL,
    Latest_Op_Expense REAL,
    Month TEXT
    )""")

cursor.execute(query)

# Creates the table by commiting the command
conn.commit()

# Imports our dataframe to the SQLite table
df.to_sql('IPO_Database', conn, if_exists='replace', index=False)

# Database export
query2 = "SELECT * FROM IPO_Database"

# Read Query2 results to a dataframe
df = pd.read_sql_query(query2, conn)

# # Export here
# df.to_csv('IPO_Database.csv', index=False)

# files.download('IPO_Database.csv')

conn.close()
files.download('IPO_Database.db')


# cursor.execute("SELECT * FROM IPO_Database WHERE Percent_Increase > 100")

# cursor.execute("SELECT * FROM IPO_Database WHERE Industry == 'Biotechnology'")

# Prints every row in the cursor execution
# for row in cursor.fetchall():
#     print(row)

# Closes the connections
# cursor.close()
# conn.close()
